In [1]:
import pandas as pd
import nltk
import re
import numpy as np
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


In [2]:
df = pd.read_csv('/content/drive/MyDrive/NLP/Fake News/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df = df.dropna()

In [5]:
X = df.drop('label', axis=1)
Y=df.label

In [6]:
X.shape

(18285, 4)

In [7]:
Y.shape

(18285,)

Vocabulary Size

In [8]:
voca_size = 50000

In [9]:
message = X.copy()

In [10]:
message.reset_index(inplace=True)

One Hot Representation

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
### Data preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0,len(message)):
  print(i)
  review = re.sub('[^a-zA-Z]', ' ' , message['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)



Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [13]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [14]:
onehot_rep = [one_hot(words,voca_size) for words in corpus]

In [15]:
onehot_rep

[[14351, 29304, 34728, 38285, 26951, 48032, 2248, 44553, 44719, 30225],
 [43358, 47486, 45830, 45027, 19949, 28265, 23637],
 [6245, 38000, 6481, 4459],
 [22897, 46420, 3677, 28571, 4128, 36842],
 [49138, 19949, 11025, 46129, 43595, 5031, 19949, 35377, 10112, 4403],
 [33357,
  44048,
  44094,
  46446,
  11463,
  5526,
  6208,
  16419,
  44437,
  48830,
  8724,
  2308,
  12599,
  13667,
  23637],
 [34316, 35157, 18590, 7360, 32091, 38197, 37454, 6170, 35041, 33890, 27556],
 [6269, 20346, 30622, 21372, 40375, 27783, 5526, 3122, 35041, 33890, 27556],
 [19983, 47272, 33744, 15842, 7781, 17542, 29560, 11903, 5526, 36339],
 [7017, 27814, 20592, 45513, 40823, 28875, 47867, 5351],
 [29299, 23396, 23983, 46121, 10847, 19737, 24730, 42313, 14917, 8691, 19774],
 [28571, 42666, 26951, 17542, 5526, 40375],
 [37154, 32918, 41988, 19998, 17432, 20489, 46797, 41988, 28234],
 [19375, 24027, 39558, 26069, 2463, 48744, 84, 35041, 33890, 27556],
 [24345, 12028, 49671, 14589, 37612, 35041, 33890, 27556],
 [



Embedding Represention



In [16]:
sen_length = 20
embadding_doc = pad_sequences(onehot_rep,padding='pre', maxlen=sen_length)
print(embadding_doc)

[[    0     0     0 ... 44553 44719 30225]
 [    0     0     0 ... 19949 28265 23637]
 [    0     0     0 ... 38000  6481  4459]
 ...
 [    0     0     0 ... 35041 33890 27556]
 [    0     0     0 ... 18950  3354 46161]
 [    0     0     0 ... 39043 48327  5839]]


In [17]:
embedding_vac_features = 40
model = Sequential()
model.add(Embedding(voca_size,embedding_vac_features,input_length=sen_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            2000000   
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 2,056,501
Trainable params: 2,056,501
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
X_final = np.array(embadding_doc)
Y_final = np.array(Y)

In [19]:
X_final.shape,Y_final.shape

((18285, 20), (18285,))

In [20]:
from sklearn.model_selection import train_test_split
X_train,x_test,Y_train,y_test = train_test_split(X_final,Y_final,test_size=0.25,random_state=42)

In [21]:
model.fit(X_train,Y_train,validation_data=(x_test,y_test),epochs=10,batch_size=100)

Epoch 1/10
138/138 [==============================] - 11s 65ms/step - loss: 0.5272 - accuracy: 0.7203 - val_loss: 0.2017 - val_accuracy: 0.9151
Epoch 2/10
138/138 [==============================] - 8s 60ms/step - loss: 0.1250 - accuracy: 0.9522 - val_loss: 0.1837 - val_accuracy: 0.9230
Epoch 3/10
138/138 [==============================] - 8s 60ms/step - loss: 0.0659 - accuracy: 0.9785 - val_loss: 0.2274 - val_accuracy: 0.9202
Epoch 4/10
138/138 [==============================] - 8s 61ms/step - loss: 0.0321 - accuracy: 0.9896 - val_loss: 0.2802 - val_accuracy: 0.9189
Epoch 5/10
138/138 [==============================] - 8s 60ms/step - loss: 0.0226 - accuracy: 0.9929 - val_loss: 0.3045 - val_accuracy: 0.9140
Epoch 6/10
138/138 [==============================] - 8s 60ms/step - loss: 0.0120 - accuracy: 0.9967 - val_loss: 0.2945 - val_accuracy: 0.9138
Epoch 7/10
138/138 [==============================] - 8s 60ms/step - loss: 0.0120 - accuracy: 0.9970 - val_loss: 0.4249 - val_accuracy: 0.913

In [22]:
y_pred = model.predict_classes(x_test)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [23]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [24]:
confusion_matrix(y_test,y_pred)

array([[2407,  193],
       [ 200, 1772]])

In [25]:
accuracy_score(y_test,y_pred)

0.9140419947506562

In [26]:
test_df = pd.read_csv('/content/drive/MyDrive/NLP/Fake News/test.csv')

In [28]:
test_df = test_df.dropna()

In [31]:
test_df.reset_index(inplace=True)

In [32]:
ps = PorterStemmer()
corpus1 = []
for i in range(0,len(test_df)):
  print(i)
  review = re.sub('[^a-zA-Z]', ' ' , test_df['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus1.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [33]:
onehot_rep = [one_hot(words,voca_size) for words in corpus]
onehot_rep

[[14351, 29304, 34728, 38285, 26951, 48032, 2248, 44553, 44719, 30225],
 [43358, 47486, 45830, 45027, 19949, 28265, 23637],
 [6245, 38000, 6481, 4459],
 [22897, 46420, 3677, 28571, 4128, 36842],
 [49138, 19949, 11025, 46129, 43595, 5031, 19949, 35377, 10112, 4403],
 [33357,
  44048,
  44094,
  46446,
  11463,
  5526,
  6208,
  16419,
  44437,
  48830,
  8724,
  2308,
  12599,
  13667,
  23637],
 [34316, 35157, 18590, 7360, 32091, 38197, 37454, 6170, 35041, 33890, 27556],
 [6269, 20346, 30622, 21372, 40375, 27783, 5526, 3122, 35041, 33890, 27556],
 [19983, 47272, 33744, 15842, 7781, 17542, 29560, 11903, 5526, 36339],
 [7017, 27814, 20592, 45513, 40823, 28875, 47867, 5351],
 [29299, 23396, 23983, 46121, 10847, 19737, 24730, 42313, 14917, 8691, 19774],
 [28571, 42666, 26951, 17542, 5526, 40375],
 [37154, 32918, 41988, 19998, 17432, 20489, 46797, 41988, 28234],
 [19375, 24027, 39558, 26069, 2463, 48744, 84, 35041, 33890, 27556],
 [24345, 12028, 49671, 14589, 37612, 35041, 33890, 27556],
 [

In [34]:
sen_length = 20
embadding_doc_test = pad_sequences(onehot_rep,padding='pre', maxlen=sen_length)
print(embadding_doc_test)

[[    0     0     0 ... 44553 44719 30225]
 [    0     0     0 ... 19949 28265 23637]
 [    0     0     0 ... 38000  6481  4459]
 ...
 [    0     0     0 ... 35041 33890 27556]
 [    0     0     0 ... 18950  3354 46161]
 [    0     0     0 ... 39043 48327  5839]]


In [35]:
y_pred = model.predict_classes(embadding_doc_test)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [40]:
print(y_pred)

[[1]
 [0]
 [1]
 ...
 [0]
 [0]
 [1]]
